In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
#!connect mssql --kernel-name sql2025 --connection-string "Server=SQL2025;TrustServerCertificate=True;Integrated Security=True"

In [ ]:
SELECT @@VERSION

In [ ]:
USE LinkedInCourseDB


Get your API Key at: https://www.nps.gov/subjects/developer/get-started.htm

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL [https://developer.nps.gov/]
WITH IDENTITY = 'HttpEndpointQueryString', SECRET = '{"api_key":"XXX"}'

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[NPS_Places]') AND type in (N'U'))
DROP TABLE [dbo].[NPS_Places]
GO
CREATE TABLE [dbo].[NPS_Places](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[ParkId] [nvarchar](100) NULL,
	[ParkName] [nvarchar](500) NULL,
	[weburl] [nvarchar](500) NULL,
	[PlaceName] [nvarchar](500) NULL,
	[Description_Short] [nvarchar](max) NULL,
	[Description_Long] [nvarchar](max) NULL,
	[Description_Summary] [nvarchar](max) NULL,
	[latitude] float NULL,
	[longitude] float NULL,
	[isOpenToPublic] [nvarchar](10) NULL,
	[Embeddings_Chunk] [nvarchar](max) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[NPS_Places_Stage]') AND type in (N'U'))
DROP TABLE [dbo].[NPS_Places_Stage]
GO
CREATE TABLE [dbo].[NPS_Places_Stage](
	[ParkId] [nvarchar](100) NULL,
	[ParkName] [nvarchar](500) NULL,
	[weburl] [nvarchar](500) NULL,
	[PlaceName] [nvarchar](500) NULL,
	[Description_Short] [nvarchar](max) NULL,
	[Description_Long] [nvarchar](max) NULL,
	[latitude] float NULL,
	[longitude] float NULL,
	[isOpenToPublic] [nvarchar](10) NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]

In [ ]:
CREATE UNIQUE CLUSTERED INDEX IDX_ID 
ON NPS_Places (ID)

In [ ]:
SET NOCOUNT ON
TRUNCATE TABLE NPS_Places_Stage
DECLARE @response NVARCHAR(max)
Declare @limit int = 1000
declare @start int = 1
declare @total int = 1
declare @url nvarchar(500)


WHILE @start <= @total
BEGIN
    SET @url = 'https://developer.nps.gov/api/v1/places?limit=' + LTRIM(STR(@limit)) + '&start=' + LTRIM(STR(@start))

    EXEC sp_invoke_external_rest_endpoint
      @url = @url,
      @credential = [https://developer.nps.gov/],
      @method = 'GET',
      @response = @response OUTPUT
    SELECT @total=[value] FROM   Openjson(Json_query(@response, '$.result')) A where [key] = 'total'
    INSERT INTO NPS_Places_Stage (
        [ParkId],
        [ParkName],
        [weburl],
        [PlaceName],
        [Description_Short],
        [Description_Long],
        [latitude],
        [longitude],
        [isOpenToPublic]
    )
    SELECT 
      id,
      relatedParkFullName,
      url,
      title,
      listingDescription,
      bodyText,
      latitude,
      longitude,
      isOpenToPublic
    FROM OPENJSON(JSON_QUERY(@response, '$.result.data'))
    WITH (
        id NVARCHAR(100) '$.id',
        url NVARCHAR(500) '$.url',
        title NVARCHAR(500) '$.title',
        listingDescription NVARCHAR(MAX) '$.listingDescription',
        bodyText NVARCHAR(MAX) '$.bodyText',
        latitude NVARCHAR(50) '$.latitude',
        longitude NVARCHAR(50) '$.longitude',
        isOpenToPublic NVARCHAR(10) '$.isOpenToPublic',
        relatedParkFullName NVARCHAR(500) '$.relatedParks[0].fullName'
    )

    SET @start = @start + @limit
END
SET NOCOUNT OFF

In [ ]:
DELETE FROM NPS_Places_Stage WHERE Latitude = 0 or Longitude = 0 or IsOpenToPublic = 0 or ParkName is NULL

In [ ]:
INSERT INTO NPS_Places ([ParkId]
      ,[ParkName]
      ,[weburl]
      ,[PlaceName]
      ,[Description_Short]
      ,[Description_Long]
      ,[latitude]
      ,[longitude]
      ,[isOpenToPublic])
SELECT [ParkId]
      ,[ParkName]
      ,[weburl]
      ,[PlaceName]
      ,[Description_Short]
      ,[Description_Long]
      ,[latitude]
      ,[longitude]
      ,[isOpenToPublic]
  FROM [NPS_Places_Stage] WHERE NOT [ParkId] in (SELECT [ParkId] FROM NPS_Places)


In [ ]:
DELETE FROM NPS_Places WHERE NOT [ParkId] in (SELECT [ParkId] FROM NPS_Places_Stage)

In [ ]:
SELECT COUNT(*) FROM NPS_Places

In [ ]:
SELECT TOP 1 * FROM NPS_Places

In [ ]:
UPDATE NPS_Places SET Embeddings_Chunk = 
   PlaceName + ' (located at: ' + isnull(ParkName,'') + ') - ' + Description_Long

In [ ]:
SELECT c.*
FROM NPS_Places p
CROSS APPLY
   AI_GENERATE_CHUNKS(source = p.Embeddings_Chunk, 
   chunk_type = FIXED, 
   chunk_size = 150, 
   enable_chunk_set_id = 1) c
WHERE id =1

In [ ]:
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[NPS_Places_Embeddings]') AND type in (N'U'))
DROP TABLE [dbo].[NPS_Places_Embeddings]
GO
CREATE TABLE [dbo].[NPS_Places_Embeddings](
    [Id] [int] IDENTITY(1,1) NOT NULL,
	[ParkId] [int] NOT NULL,
	[Chunk] [nvarchar](max) NULL,
	[Embeddings] [vector](768) NULL,
	[Embeddings_Calculated] datetime NULL
) ON [PRIMARY] TEXTIMAGE_ON [PRIMARY]

In [ ]:
INSERT INTO NPS_Places_Embeddings (ParkId,Chunk)
SELECT id ParkId,c.Chunk
FROM NPS_Places p 
CROSS APPLY
   AI_GENERATE_CHUNKS(source = p.Embeddings_Chunk, 
   chunk_type = FIXED, 
   -- Deliberately too high chunk_size!
   chunk_size = 2500, 
   enable_chunk_set_id = 1) c
WHERE NOT ID in (SELECT ParkId FROM NPS_Places_Embeddings)

In [ ]:
CREATE EVENT SESSION [AIMonitor] ON SERVER 
ADD EVENT sqlserver.ai_generate_embeddings_stats,
ADD EVENT sqlserver.ai_generate_embeddings_summary,
ADD EVENT sqlserver.external_rest_endpoint_invocation_attempt,
ADD EVENT sqlserver.external_rest_endpoint_summary
--WITH (STARTUP_STATE=ON)
GO
--ALTER EVENT SESSION [AIMonitor] ON SERVER STATE = START

In [ ]:
SET NOCOUNT ON
DECLARE 
    @id NVARCHAR(100),
    @chunk NVARCHAR(MAX)

DECLARE row_cursor CURSOR FOR
    SELECT Id, Chunk 
    FROM NPS_Places_Embeddings 
    WHERE embeddings IS NULL
    ORDER BY ID

OPEN row_cursor

FETCH NEXT FROM row_cursor INTO @id, @chunk

WHILE @@FETCH_STATUS = 0
BEGIN
    UPDATE NPS_Places_Embeddings
    SET Embeddings_Calculated = getdate(),
    Embeddings = AI_GENERATE_EMBEDDINGS(Chunk USE MODEL ollama)
    WHERE Id = @id

    FETCH NEXT FROM row_cursor INTO @id, @chunk
END

CLOSE row_cursor
DEALLOCATE row_cursor
SET NOCOUNT OFF

In [ ]:
DROP EVENT SESSION [AIMonitor] ON SERVER 

In [ ]:
DECLARE @response NVARCHAR(max)

EXEC Sp_invoke_external_rest_endpoint
  @url = N'https://nominatim.openstreetmap.org/search?q=Boston,MA&format=json',
  @method = 'GET',
  @response = @response output

SELECT Json_value(value, '$.lat') lat,Json_value(value, '$.lon') lon FROM OPENJSON(JSON_QUERY(@response, '$.result'))

In [ ]:
CREATE OR ALTER FUNCTION dbo.GetNearbyPlaces
(
    @lat FLOAT,
    @lon FLOAT,
    @radius FLOAT 
)
RETURNS TABLE
AS
RETURN
(
    SELECT * FROM (
    SELECT 
        *,
        3959 * ACOS(
            COS(RADIANS(@lat)) * COS(RADIANS(latitude)) *
            COS(RADIANS(longitude) - RADIANS(@lon)) +
            SIN(RADIANS(@lat)) * SIN(RADIANS(latitude))
        ) AS DistanceInMiles
    FROM NPS_Places
    WHERE 
        latitude IS NOT NULL AND 
        longitude IS NOT NULL) A 
        WHERE DistanceInMiles <= @radius
)

In [ ]:
DECLARE @radius Float = 10
DECLARE @place NVARCHAR(100) = 'Boston,MA'
DECLARE @lat Float
DECLARE @lon Float
DECLARE @response NVARCHAR(max)
DECLARE @url NVARCHAR(max) = N'https://nominatim.openstreetmap.org/search?q=' + @place + '&format=json'
EXEC Sp_invoke_external_rest_endpoint
  @url = @url,
  @method = 'GET',
  @response = @response output

SELECT @lat=Json_value(value, '$.lat') ,@lon=Json_value(value, '$.lon')  FROM OPENJSON(JSON_QUERY(@response, '$.result'))

SELECT TOP 10 ParkName,PlaceName,Description_Summary FROM dbo.GetNearbyPlaces(@lat, @lon, @radius)

In [ ]:
DECLARE @radius Float = 250
DECLARE @place NVARCHAR(100) = 'Raleigh,NC'
DECLARE @search_text NVARCHAR(MAX) = 'Aviation'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)
DECLARE @lat Float
DECLARE @lon Float
DECLARE @response NVARCHAR(max)
DECLARE @url NVARCHAR(max) = N'https://nominatim.openstreetmap.org/search?q=' + @place + '&format=json'
EXEC Sp_invoke_external_rest_endpoint
  @url = @url,
  @method = 'GET',
  @response = @response output

SELECT @lat=Json_value(value, '$.lat') ,@lon=Json_value(value, '$.lon')  FROM OPENJSON(JSON_QUERY(@response, '$.result'))

SELECT Top 3 ParkName,PlaceName,Distance FROM NPS_Places a INNER JOIN (
SELECT b.ParkId,
MIN(vector_distance('cosine', @search_vector, b.embeddings)) AS distance 
FROM dbo.GetNearbyPlaces(@lat, @lon, @radius) a 
INNER JOIN NPS_Places_Embeddings b on a.id = b.parkid
WHERE b.embeddings is not null
GROUP BY b.ParkId) b ON a.ID = b.ParkId order by distance

In [ ]:
CREATE OR ALTER PROCEDURE dbo.FindNearbyPlacesByVectorSearch
    @place NVARCHAR(100),
    @search_text NVARCHAR(MAX),
    @radius FLOAT
AS
BEGIN
    SET NOCOUNT ON

    DECLARE 
        @lat FLOAT,
        @lon FLOAT,
        @response NVARCHAR(MAX),
        @search_vector VECTOR(768),
        @url NVARCHAR(MAX)

    SET @search_vector = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)

    SET @url = N'https://nominatim.openstreetmap.org/search?q=' + @place + '&format=json'

    EXEC sp_invoke_external_rest_endpoint
        @url = @url,
        @method = 'GET',
        @response = @response OUTPUT

    SELECT 
        @lat = TRY_CAST(JSON_VALUE(value, '$.lat') AS FLOAT),
        @lon = TRY_CAST(JSON_VALUE(value, '$.lon') AS FLOAT)
    FROM OPENJSON(JSON_QUERY(@response, '$.result'))

    IF @lat IS NULL OR @lon IS NULL
    BEGIN
        RAISERROR('Failed to resolve location: %s', 16, 1, @place)
        RETURN
    END

    SELECT Top 3 ParkName,PlaceName,Description_Summary,Distance 
    FROM NPS_Places a INNER JOIN (
            SELECT b.ParkId,MIN(vector_distance('cosine', @search_vector, b.embeddings)) AS distance 
            FROM dbo.GetNearbyPlaces(@lat, @lon, @radius) a 
            INNER JOIN NPS_Places_Embeddings b on a.id = b.parkid
            WHERE b.embeddings is not null
            GROUP BY b.ParkId) b 
    ON a.ID = b.ParkId order by distance
END


In [ ]:
exec FindNearbyPlacesByVectorSearch 'Orlando,FL','Space travel',150

In [ ]:
CREATE OR ALTER PROCEDURE dbo.CreateShortDescription
    @Parkid int
AS
BEGIN
DECLARE 
    @chunk NVARCHAR(MAX),
    @prompt NVARCHAR(MAX),
    @model NVARCHAR(250) = N'mistral',
    @payload NVARCHAR(MAX),
    @response NVARCHAR(MAX),
    @summary NVARCHAR(MAX)
IF EXISTS (SELECT 1 FROM NPS_Places WHERE ID = @Parkid AND Description_Summary IS NULL)
BEGIN
    SELECT @Chunk=embeddings_chunk FROM NPS_Places WHERE ID = @ParkId
    SET @prompt = N'Please summarize the following description in one sentence: ' + ISNULL(@chunk, '')
    SET @prompt = REPLACE(@prompt, '\', '\\') 
    SET @prompt = REPLACE(@prompt, '"', '\"') 
    SET @payload = N'{"model":"' + @model + '","prompt":"' + @prompt + '","stream": false }'

    EXEC sp_invoke_external_rest_endpoint
        @url = N'https://ai-gpu.lab.bwdemo.io:443/api/generate',
        @payload = @payload,
        @timeout = 230,
        @response = @response OUTPUT

    SELECT @summary = [value]
    FROM OPENJSON(JSON_QUERY(@response, '$.result'))
    WHERE [key] = 'response'

    UPDATE NPS_Places
    SET Description_Summary = @summary
    WHERE Id = @Parkid
END
END

In [ ]:
SELECT Description_Summary FROM NPS_Places WHERE ID = 1

In [ ]:
EXEC CreateShortDescription 1

In [ ]:
SELECT Description_Summary FROM NPS_Places WHERE ID = 1

In [ ]:
SET NOCOUNT ON
DECLARE 
    @id int
DECLARE row_cursor CURSOR FOR
    SELECT id
    FROM NPS_Places 
    WHERE Description_Summary IS NULL 
    ORDER BY ID
OPEN row_cursor

FETCH NEXT FROM row_cursor INTO @id

WHILE @@FETCH_STATUS = 0
BEGIN
    EXEC CreateShortDescription @id
    FETCH NEXT FROM row_cursor INTO @id
END

CLOSE row_cursor
DEALLOCATE row_cursor
SET NOCOUNT OFF

In [ ]:
SELECT count(*) FROM NPS_Places WHERE Description_Summary IS NOT NULL

In [ ]:
exec FindNearbyPlacesByVectorSearch 'Orlando,FL','Space travel',150

In [ ]:
CREATE OR ALTER PROCEDURE dbo.FindNearbyPlacesByVectorSearch
    @place NVARCHAR(100),
    @search_text NVARCHAR(MAX),
    @radius FLOAT
AS
BEGIN
    SET NOCOUNT ON

    DECLARE 
        @lat FLOAT,
        @lon FLOAT,
        @response NVARCHAR(MAX),
        @search_vector VECTOR(768),
        @url NVARCHAR(MAX)

    SET @search_vector = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)

    SET @url = N'https://nominatim.openstreetmap.org/search?q=' + @place + '&format=json'

    EXEC sp_invoke_external_rest_endpoint
        @url = @url,
        @method = 'GET',
        @response = @response OUTPUT

    SELECT 
        @lat = TRY_CAST(JSON_VALUE(value, '$.lat') AS FLOAT),
        @lon = TRY_CAST(JSON_VALUE(value, '$.lon') AS FLOAT)
    FROM OPENJSON(JSON_QUERY(@response, '$.result'))

    IF @lat IS NULL OR @lon IS NULL
    BEGIN
        RAISERROR('Failed to resolve location: %s', 16, 1, @place)
        RETURN
    END

    SELECT Top 3 a.Id,Distance,Description_Summary INTO #Result
    FROM NPS_Places a INNER JOIN (
            SELECT b.ParkId,MIN(vector_distance('cosine', @search_vector, b.embeddings)) AS distance 
            FROM dbo.GetNearbyPlaces(@lat, @lon, @radius) a 
            INNER JOIN NPS_Places_Embeddings b on a.id = b.parkid
            WHERE b.embeddings is not null
            GROUP BY b.ParkId) b 
    ON a.ID = b.ParkId 
    order by distance

    
DECLARE 
    @id int
DECLARE row_cursor CURSOR FOR
    SELECT id
    FROM #Result 
    WHERE Description_Summary IS NULL
    ORDER BY ID
OPEN row_cursor

FETCH NEXT FROM row_cursor INTO @id

WHILE @@FETCH_STATUS = 0
BEGIN
    EXEC CreateShortDescription @id
    FETCH NEXT FROM row_cursor INTO @id
END

CLOSE row_cursor
DEALLOCATE row_cursor

    
    SELECT ParkName,PlaceName,b.Description_Summary,Distance FROM #Result a 
    INNER JOIN NPS_Places b ON a.Id = b.Id
    order by distance
END


In [ ]:
exec FindNearbyPlacesByVectorSearch 'Orlando,FL','Space travel',150